In [ ]:
import numpy as np
import gensim
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import torch.optim as optim
from tqdm import tqdm

# Part a

## Loading the Dataset and replacing positive with 1 and negative with 0

In [ ]:
dir_path = pathlib.Path('IMDB_Dataset/IMDB Dataset.csv')

In [ ]:
dir_path.exists()

In [ ]:
data_df = pd.read_csv(dir_path)

In [ ]:
data_df['sentiment']=data_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
data_df.head()

## Splitting into train test val

In [ ]:
# Shuffling the data
df = data_df.sample(frac=1,random_state=42).reset_index(drop=True)
df_train,df_val,df_test = df[:40000],df[40000:45000],df[45000:]

## Scatter plot of word-embeddings

In [ ]:
# Load glove embeddings
import gensim.downloader as api
glove_vectors = api.load('glove-wiki-gigaword-100')

In [ ]:
# To deal with cluttering due to a lot of words in the ten sentences I have used only two sentences from the validation set instead of 10
#TODO : Change it to 10 sentences if no reply is received
val_words = dict()
for sentence in df_val['review'][:2]:
    for word in sentence.split():
        if word.lower() in glove_vectors:
            val_words[word.lower()] = glove_vectors[word.lower()]

In [ ]:
from sklearn.decomposition import PCA
pca_obj = PCA(n_components=2)
pca_obj.fit(list(val_words.values()))

In [ ]:
# Transforming the embeddings via pca and storing them in a dictionary
val_pca_words = dict()
for key,embedding in val_words.items():
    val_pca_words[key] = pca_obj.transform([embedding])[0]

In [ ]:

# Extracting x and y coordinates and corresponding words
x_coords = [val_pca_words[word][0] for word in val_pca_words]
y_coords = [val_pca_words[word][1] for word in val_pca_words]
words = list(val_pca_words.keys())

# Creating the scatter plot
plt.figure(figsize=(20, 20))
plt.scatter(x_coords, y_coords, alpha=0.5)

# Annotating the words
for i, word in enumerate(words):
    plt.annotate(word, (x_coords[i], y_coords[i]), fontsize=14, alpha=0.7)

plt.title("Word Embeddings Scatter Plot")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.grid(True)
plt.show()

# TODO: Write the semantic clustering in the answer
# The articles are clustered in the left portion of the plot along with common helping-verbs like was, is, are, etc.
# The adjective words are clustered in the right portion of the plot. The words like "wonderful", "innocent", "forget" are clustered together.
# Pronouns I and you are clustered together. at the bottom left 
# The words like "movie", "film", "story" are clustered together.

## Creating features from word to vec for the second part

In [ ]:
# Function that creates a sentence representation words not in glove are dropped.
def create_sentence_representation(sentence,glove_vectors):
    sentence_representation = []
    sentence = sentence.lower()
    for word in sentence.split():
        if word in glove_vectors:
            sentence_representation.append(glove_vectors[word])
    return sentence_representation

In [ ]:
X_train = [torch.tensor(np.array(create_sentence_representation(sentence,glove_vectors))) for sentence in df_train['review']]
X_val = [torch.tensor(np.array(create_sentence_representation(sentence,glove_vectors))) for sentence in df_val['review']]
X_test = [torch.tensor(np.array(create_sentence_representation(sentence,glove_vectors))) for sentence in df_test['review']]
y_train = df_train['sentiment'].values
y_val = df_val['sentiment'].values
y_test = df_test['sentiment'].values

# Part b

## TODO : Mention the initial Learning rate in the report

## Creating a dataloader

In [ ]:
from torch.nn.utils.rnn import pad_sequence
def my_collate(batch):
    len_list = []
    x_list = []
    y_list = []
    for x,y in batch:
        len_list.append(x.shape[0])
        x_list.append(x)
        y_list.append(y)
    x_list = pad_sequence(x_list,batch_first=True) # batch,max_seq_len,100
    y_list = torch.tensor(y_list)
    len_list = torch.tensor(len_list)
    return x_list,y_list.float(),len_list


In [ ]:
def create_dataloder(X,y,batch_size):
    dataset = list(zip(X,y))
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True,collate_fn=my_collate)
    return dataloader

In [ ]:
batch_size = 32

In [ ]:
train_dl = create_dataloder(X_train,y_train,batch_size)
val_dl = create_dataloder(X_val,y_val,batch_size)
test_dl = create_dataloder(X_test,y_test,batch_size)

## Creating and Training LSTM Model

In [ ]:
class lstm_model_avg_pool(nn.Module):
    def __init__(self,hidden_size=256,num_layers=2):
        super().__init__()
        self.classifier = nn.Linear(hidden_size,1)
        self.lstm = nn.LSTM(100,hidden_size,num_layers,batch_first=True)
    
    def make_mask(self,len_list,device):
        batch_size = len_list.shape[0]
        max_len = len_list.max()
        mask = len_list.reshape(batch_size,1) > torch.arange(max_len).reshape(1,max_len).to(device) # batch,max_len
        return mask.reshape(batch_size,max_len,1)
    def forward(self,x,len_list,device):
        '''
        len_list : batch to be used to make a mask
        '''
        out,_ = self.lstm(x) #  batch,seq_len,hidde_size
        mask = self.make_mask(len_list,device)
        out = out*mask # batch,seq_len,hidden_size
        out_pool = out.sum(dim=1)/len_list.reshape(-1,1) # batch,hidden_size
        return self.classifier(out_pool) # batch,1

In [ ]:
# An abstract class that will be used to train the models inspired from the assignments of NLP course
class ModelTrainer:
    def __init__(self,model,train_dl,val_dl,optimizer=None,batch_size=32,epochs=25):
        self.model = model
        if optimizer is None:
            self.optimizer = optim.Adam(model.parameters(),lr=1e-3)
        else:
            self.optimizer = optimizer
        self.criterion = nn.BCEWithLogitsLoss()
        self.epochs = epochs
        self.train_loss = {}
        self.val_loss = {}
        self.batch_size = batch_size
        self.train_dl = train_dl
        self.val_dl = val_dl
        self.steps = 0
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
    def train(self):
        for epoch in tqdm(range(self.epochs)):
                for xb,yb,len_list in train_dl:
                    self.model.train()
                    self.optimizer.zero_grad()
                    xb,yb,len_list = xb.to(self.device),yb.to(self.device),len_list.to(self.device)
                    y_pred = self.model(xb,len_list,self.device)
                    loss = self.criterion(y_pred,yb.unsqueeze(1))
                    loss.backward()
                    self.optimizer.step()
                    self.steps += 1
                self.train_loss[epoch] = self.get_validation_loss(self.train_dl)
                self.val_loss[epoch] = self.get_validation_loss(self.val_dl)
                tqdm.write(f'Epoch : {epoch} Train Loss : {self.train_loss[epoch]} Validation Loss : {self.val_loss[epoch]}')                    
                
    def get_validation_loss(self,set_dl):
        self.model.eval()
        val_dl = set_dl
        with torch.no_grad():
            val_loss = 0
            total_len = 0
            for xb,yb,len_list in val_dl:
                xb,yb,len_list = xb.to(self.device),yb.to(self.device),len_list.to(self.device)
                y_pred = self.model(xb,len_list,self.device)
                loss = self.criterion(y_pred,yb.unsqueeze(1))
                val_loss += loss.item()*len(yb)
                total_len += len(yb)
            return val_loss/total_len
        
    def plot_losses(self,title_str):
        plt.plot(list(self.train_loss.keys()),list(self.train_loss.values()),label='Train Loss',marker='*')
        plt.plot(list(self.val_loss.keys()),list(self.val_loss.values()),label='Validation Loss',marker='o')
        plt.title(title_str)
        plt.xlabel('Number of Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(title_str+'.png')
        plt.show()
    
    def get_test_accuracy(self,test_dl):
        self.model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for xb,yb,len_list in test_dl:
                xb,yb,len_list = xb.to(self.device),yb.to(self.device),len_list.to(self.device)
                y_pred = self.model.forward(xb,len_list,self.device)
                y_pred = torch.where(y_pred>0,1,0)
                correct += (y_pred == yb.unsqueeze(1)).sum().item()
                total += yb.shape[0]
            return correct/total

In [ ]:
lstm_model_avg_pool_obj = lstm_model_avg_pool()
model_trainer = ModelTrainer(lstm_model_avg_pool_obj,train_dl=train_dl,val_dl=val_dl)
model_trainer.train()

In [ ]:
print(f"Test Accuracy lstm avg pool: {model_trainer.get_test_accuracy(test_dl)}")
model_trainer.plot_losses('Train and Validation Losses LSTM Avg Pool')
del(lstm_model_avg_pool_obj)
del(model_trainer)

# Part c

## Creating and training a attention based pooling model

In [ ]:
class lstm_model_attention(nn.Module):
    def __init__(self,hidden_size=256,num_layers=2):
        super().__init__()
        self.classifier = nn.Linear(hidden_size,1)
        self.lstm = nn.LSTM(100,hidden_size,num_layers,batch_first=True)
        self.V = nn.Linear(hidden_size,1)
        self.W = nn.Linear(hidden_size,hidden_size)
    def attention(self,out):
        '''
        out : batch,max_seq_len,hidden_size
        Implements attention socres as v.Tanh(W*out+b)
        '''
        unnorm_attn_scores = self.V(torch.tanh(self.W(out)))
        return unnorm_attn_scores # batch,max_seq_len,1
    def make_mask(self,len_list,device):
        batch_size = len_list.shape[0]
        max_len = len_list.max()
        mask = len_list.reshape(batch_size,1) > torch.arange(max_len).reshape(1,max_len).to(device) # batch,max_len
        return mask.reshape(batch_size,max_len,1)
            
    def forward(self,x,len_list,device):
        out,_ = self.lstm(x) # batch,max_seq_len,hidde_size
        unnorm_scores = self.attention(out) # batch,max_seq_len,1
        mask = self.make_mask(len_list,device)
        attn_scores = torch.where(mask,unnorm_scores,-float('inf')) # batch,max_seq_len,1
        attn_scores = torch.softmax(attn_scores,dim=1).reshape(-1,1,len_list.max()) # batch,1,max_seq_len
        out_context = attn_scores@out # batch,1,hidden_size
        final_out = self.classifier(out_context).reshape(-1,1) # batch,1
        return final_out

In [ ]:
lst_model_attention_obj = lstm_model_attention()
model_trainer = ModelTrainer(lst_model_attention_obj,train_dl=train_dl,val_dl=val_dl)
model_trainer.train()

In [ ]:
print(f"Test Accuracy lstm attention: {model_trainer.get_test_accuracy(test_dl)}")
model_trainer.plot_losses('Train and Validation Losses LSTM Attention')
del(lst_model_attention_obj)
del(model_trainer)
#TODO: Write that tanh attention is used in the answer

# Part d

## Implementing and training a transformer encoder block

In [ ]:
class transformer_encoder_model(nn.Module):
    def __init__(self,hidden_size=256):
        super().__init__()
        self.classifier = nn.Linear(hidden_size,1)
        self.Q = nn.Linear(hidden_size,hidden_size,bias=False)
        self.K = nn.Linear(hidden_size,hidden_size,bias=False)
        self.V = nn.Linear(hidden_size,hidden_size,bias=False)
    
    def make_mask(self,len_list,device):
        batch_size = len_list.shape[0]
        max_len = len_list.max()
        mask = len_list.reshape(batch_size,1) > torch.arange(max_len).reshape(1,max_len).to(device) # batch,max_len
        return mask.reshape(batch_size,max_len,1)


    def forward(self,x,len_list,device):
        query= self.Q(x) # batch,max_seq_len,hidden_size
        key = self.K(x) 
        vals = self.V(x)
        mask = self.make_mask(len_list,device)
        attn_scores = query@key.transpose(1,2) # batch,max_seq_len,max_seq_len
        scaled_attn_scores = attn_scores/np.sqrt(query.shape[-1]) # batch,max_seq_len,max_seq_len
        scaled_attn_scores = scaled_attn_scores.transpose(1,2) # batch,max_seq_len,max_seq_len
        masked_attn_scores = torch.where(mask,scaled_attn_scores,-float('inf')).transpose(1,2) # batch,max_seq_len,max_seq_len
        normalized_attn_scores = torch.softmax(masked_attn_scores,dim=2)
        out = normalized_attn_scores@vals # batch,max_seq_len,hidden_size
        out_masked = out*mask
        out_masked_comb = out_masked.sum(dim=1)/len_list.reshape(-1,1)
        return self.classifier(out_masked_comb)

In [ ]:
transformer_encoder_model_obj = transformer_encoder_model()
model_trainer = ModelTrainer(transformer_encoder_model_obj,train_dl=train_dl,val_dl=val_dl)
model_trainer.train()

In [ ]:
print(f"Test Accuracy transformer encoder: {model_trainer.get_test_accuracy(test_dl)}")
model_trainer.plot_losses('Train and Validation Losses Transformer Encoder')